# Lab 11: Working with FITS files
Based on the astropy tutorial http://docs.astropy.org/en/stable/io/fits/

## FITS Files
FITS (Flexible Image Transport System) is a portable file standard widely used in the astronomy community to store images and tables. A fits file generally contains two major pieces of information the **Header** and **Data/Table**.

## Loading a FITS File

FITS files can store more than one image/table and header. An HDU (Header Data Unit) is the highest level component of the FITS file structure, consisting of a header and (typically) a data array or table. When you open a fits file, you get a list of HDUs. YOu can see information about each HDU by using .info()

In [ ]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.coordinates import SkyCoord
# change some default plotting parameters
import matplotlib as mpl
mpl.rcParams['image.origin'] = 'lower'
mpl.rcParams['image.cmap'] = 'Greys_r'
%matplotlib inline

In [ ]:
hdulist = fits.open('data/aa_aql0007_raw.fits')
hdulist.info()
#Use hdulist.close() when you are done with the file

## FITS Header
If you are unfamiliar with the FITS file you can always look at the header. Header information is stored in the .header attribute for each HDU in your list.

In [ ]:
prihdr = hdulist[0].header
print(repr(prihdr))

The Header is filled with keyword value comment groups. You can get a list of the keys, and pick out specific keys if you want. You can reference them by name or by location.

In [ ]:
print(list(prihdr.keys()))
print("My Object: {}".format(prihdr['OBJECT']))
print("My Object: {}".format(prihdr[25]))

## Adding to or Updating a Header.
Headers are stored as dictionaries, so you can create a new value or update a value just by an assignment operation.  You can update the value and the comment at the same time by using a tuple.


Be careful about **Comment or History keywords**. There tend to be multiple of them in headers, so you have to reference them by location. Also, when you assign a comment or history it automatically creates a new comment block.

In [ ]:
prihdr['Num_Monk'] = (20,"Fun Times!")
print(list(prihdr.keys()))

## Accessing Image Data
The image data is stored separately from the header in each HDU.

In [ ]:
image_data = hdulist[0].data
print(image_data[10,10])

## Displaying Data
There are two ways to display an image. The first is using the command line tool Ds9. At the command prompt type ds9 imagename. Ds9 is a powerful tool for analyzing image data. It can handle multiple hdu and has a useful interface. 

You can also view the image in python.

In [ ]:
plt.figure()
plt.imshow(image_data)
plt.colorbar()
plt.show()

We need to fix the min and max values to something more reasonable.

In [ ]:
from astropy.visualization import ZScaleInterval
interval = ZScaleInterval()
(imin,imax) = interval.get_limits(image_data)
plt.imshow(image_data, vmin=imin,vmax=imax)
plt.colorbar()
plt.show()

## Getting Values
The image arrays are stored an 2d numpy arrays, so they can be manipulated in the same way. **Be careful**, the x and y in astropy image data are reversed compared to ds9! Also, ds9 starts at pixel [1,1] and astropy starts at [0,0]. I can cut-out regions using [miny:maxy,minx:maxx].

In [ ]:
small_image = image_data[0:130,0:220]
print(small_image.shape)
(imin,imax) = interval.get_limits(small_image)
plt.imshow(small_image, vmin=imin,vmax=imax)
plt.colorbar()
plt.show()

In [ ]:
print(image_data[61,60])

## Archival Data
We often want to make finder chart, or otherwise get data that we can work with from archival sources. One image source which covers the whole sky is the Digitized Sky Survey.

In [ ]:
from astroquery.skyview import SkyView

In [ ]:
#Returns a list of HDUList
#In this case there is only one element
image_list = SkyView.get_images(position='SW And', survey=['DSS'],pixels=800)
sw_and_hdulist = image_list[0]

In [ ]:
sw_and_hdulist.info()

In [ ]:
print(repr(sw_and_hdulist[0].header))

In [ ]:
(imin,imax) = interval.get_limits(sw_and_hdulist[0].data)
plt.imshow(sw_and_hdulist[0].data, vmin=imin,vmax=imax)
plt.colorbar()
plt.show()

## Save your results
You can save your results by giving the function fits.writeto() your data and an optional header.

In [ ]:
sw_and_hdulist.writeto('data/sw_and.fits',overwrite=True)

## World Coordinate Systems
In many images, a World Coordindate System (WCS) is provided. A WCS transforms from pixel space to RA/DEC space and vice-versa. All images from Skyserver have a WCS

In [ ]:
from astropy.wcs import WCS
#Get the WCS from the header
wcs = WCS(sw_and_hdulist[0].header)
wcs.printwcs()

## Converting between RA/DEC and Pixel Space
You can use the `all_pix2world()` method to transform a pixel location into a RA and DEC both in degrees. You can use `all_world2pix()` to convert RA and DEC both in degrees into a pixel coordinate.

In [ ]:
#all_pix2world(xpixel, ypixel, origin)
#origin is equal to 0 for a numpy array.
ra, dec = wcs.all_pix2world(365, 533, 0)
print(ra, dec)
xpix, ypix = wcs.all_world2pix(ra,dec,0)
print(xpix,ypix)

## Get the coordiates of SW And
I can just you the `from_name()` method to get the coordinates of SW And. I could also just create a Skycoord object by typing in coordinates.

In [ ]:
sw_and_center = SkyCoord.from_name('SW And')

In [ ]:
xpix, ypix = wcs.all_world2pix(sw_and_center.ra,sw_and_center.dec,0)
(imin,imax) = interval.get_limits(sw_and_hdulist[0].data)
plt.imshow(sw_and_hdulist[0].data, vmin=imin,vmax=imax)
plt.scatter(xpix, ypix, s=300,edgecolor='yellow', facecolor='none')
plt.text(280,500,'SW And',color='yellow',fontsize=18)
plt.colorbar()
plt.show()

## WCS Axis Projections and Transformations
You can create an axis for your image that contains information about which pixel has what RA and DEC. Below I create an axis, `ax`, which then I can use the `get_transform()` function to tell my plotting to use RA, DEC instead of pixel coordinates.

In [ ]:
ratext,dectext = wcs.all_pix2world(280, 500, 0)

ax = plt.subplot(projection=wcs)
(imin,imax) = interval.get_limits(sw_and_hdulist[0].data)
plt.imshow(sw_and_hdulist[0].data, vmin=imin,vmax=imax)
plt.colorbar()
plt.text(ratext,dectext,'SW And',color='yellow',fontsize=18, transform=ax.get_transform('fk5'))
plt.scatter(sw_and_center.ra,sw_and_center.dec, transform=ax.get_transform('fk5'), s=300,
           edgecolor='yellow', facecolor='none')
plt.show()

## Lab 11: Now it is your turn
Please answer the following questions, then print them off and turn them in. You don't need to print the whole notebook. Only print the pages starting from here.

Name:

**Q1: Convert the RA and DEC for the object in this image into degrees, and create two new keys ra_deg and dec_deg and update the header of `hdulist[0]`.**

**Q2: What pixel in ds9 has the same value as 61,60 in `hdulist[0]`?**

**Answer:**

**Q3: Get a 400x400 pixel image of M57 and display it using `imshow()`.**

**Q4: What is the RA and Dec of pixel [302, 150] on the M57 image?**

**Q5: Draw a circle around coordinate RA = 18:53:16.87 DEC = 32:58:21.5 on the M57 image?**